<a href="https://colab.research.google.com/github/akash80e/Deep-Learning/blob/master/Pytorch_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <center> CSCI 6516 : PyTorch Basics</center>

This is a tutorial on fundamental ideas of PyTorch. This notes is just a starting point. You are encouraged to find out more. The ideal use of this notebook would be as a reference which you keep updating as you learn new things. 

<b>Credits:</b> Content presented in this tutorial is what I have learnt from my teachers, friends and the python community on stackoverflow. 


- PyTorch vs Numpy
- Compute Graph
  * Directed Acyclic Graph
  * Nodes having no parent nodes - Leaf Nodes in PyTorch [Inputs, Parameters, Expected Outputs]

In [0]:
import torch
import torch.nn.functional as F
print(torch.__version__)

import numpy as np

1.5.0+cu101


## Tensor - basic unit of representation

* Tensor:
    * [Formal] An nth-rank tensor in  m-dimensional space is a mathematical object that has n indices and m^n components and obeys certain transformation rules.
    * [Informal] An n-dimensional matrix. 

In [0]:
torch.FloatTensor(np.random.randn(10,5))

### Transpose, View (Reshape)

In [0]:
x = torch.LongTensor(
    [
       [
           [1,2],
           [3,4]
       ],

       [
           [5,6],
           [7,8]
       ],

       [
           [9,10],
           [11,12]
       ]
    ])
x_2d = torch.LongTensor([[1,2],[3,4]])

In [0]:
x.shape

<b> Transpose </b>

.t , .transpose and .permute

In [0]:
x.transpose(dim0=1,dim1=2)

In [0]:
x.permute(0,1,2)

<b> Reshape/View </b>

In [0]:
x.reshape(-1,1)

In [0]:
x.reshape(1,-1)

<b> Internals of Transpose </b>

In [0]:
# To Do

#### Exercise

Reshape x into a 2D matrix such that column 1 has [1,2,3,4]; column 2 has [5,6,7,8] and column 3 has [9,10,11,12]. 

Challenge: Can you give another way of achieving the same? 

In [0]:
x

## Arithmetic operations 

<b> Addition/Subtraction </b>

In [0]:
x

In [0]:
x + 1

In [0]:
x + x

<b> Multiplication </b>

In [0]:
x_2d * 5

In [0]:
x_2d * x_2d

In [0]:
torch.matmul(x_2d,x_2d)

<b> Division </b>

In [0]:
x_2d/3.0

In [0]:
x_2d/x_2d

<b> Broadcasting </b> <br>
Broadcasting is "auto-correction" of one of the arguments in order to bring them to the correct dimensions. 
* Replicates the 1/more dimensions of one or both the tensors to match them.
* From Scipy:
<pre>
When operating on two arrays, NumPy compares their shapes element-wise. It starts with 
the trailing dimensions, and works its way forward. Two dimensions are compatible when:
    * they are equal, or
    * one of them is 1
</pre>
* Makes a best effort to auto-correct; throws error if it cannot. 
* Not applicable to all operators.

In [0]:
x_2d

In [0]:
y = np.array([[1,2]])
print(y.shape)
x_2d + y

In [0]:
y = np.array([[1],[2]])
print(y.shape)
x_2d + y

In [0]:
y = np.array([[1,2]])
print(y.shape)
x * y

In [0]:
y = np.array([[1],[2]])
print(y.shape)
x_2d * y

In [0]:
y = np.array([[1,2]])
print(y.shape)
x_2d / y

In [0]:
y = np.array([[1],[2]])
print(y.shape)
x_2d / y

<b>Perils of broadcasting</b>

In [0]:
p = torch.FloatTensor([[1],[2],[3]])
q = torch.FloatTensor([[4,5,6]])
print(p.shape,q.shape)
p+q

## Idea of dim

In [0]:
x_2d

In [0]:
F.softmax(x_2d.float(),dim=1)

In [0]:
F.softmax(x_2d.float(),dim=0)

## Unary Tensor functions

sum, mean, max, min, etc

In [0]:
x_2d

In [0]:
x_2d.sum()

In [0]:
x_2d.sum(dim=0)

In [0]:
x_2d.sum(dim=1)

In [0]:
x_2d.sum(dim=1,keepdims=True)

<b> Exercise </b> <br>
Find the sum of each of the 2x2 matrices of x.

In [0]:
x

Other important/useful functions:
- torch.log
- torch.abs
- torch.exp
- torch.cat
- torch.stack
- torch.where
- F.leaky_relu, F.sigmoid, F.tanh, F.conv*, etc

## Autograd

Derivatives: retain_graph, create_graph

In [0]:
x = torch.FloatTensor([1]).requires_grad_(True)
y = torch.FloatTensor([2]).requires_grad_(True)
f = x**2 + y**2


Leaf Tensors and Optimizers

In [0]:
w = torch.tensor([[1.0]],requires_grad=True)
optim = torch.optim.SGD([w],lr=0.1)

Logistic Regression Training

In [0]:
%matplotlib inline
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from matplotlib.pyplot import xlim

# Create linearly separable data
x,y = make_blobs(n_samples=1000, n_features=2, centers=[[-4,-2],[4,2]], cluster_std=1, shuffle=True)
colors = np.array(['red','green'])
plt.scatter(x[:, 0], x[:, 1],
            color = colors[y].tolist(), marker='.')
xmin, xmax = xlim()

In [0]:
class LogisticRegression(torch.nn.Module):
    def __init__(self,input_dimensions):
        super(LogisticRegression, self).__init__()
        
        # Define trainable parameters
        self.w = torch.nn.Parameter(torch.randn(input_dimensions,1))
        self.b = torch.nn.Parameter(torch.zeros(1))
        
    def train(self,X,Y):
        """
        X: an array of shape [*, input_dimensions]
        Y: an array of shape [*,1]
        """
        X = torch.FloatTensor(X).view(-1,self.w.shape[0])
        Y = torch.FloatTensor(Y).view(-1,1)
        
        print("Training...")

        optim = torch.optim.SGD([self.w,self.b],lr=0.1)
        
        for i in range(50):
            pred = X@self.w + self.b
            pred = F.sigmoid(pred)
            
            cost = None # Cross-entropy loss
            reg = None  # L2 regularization 

            optim.zero_grad()
            cost.backward()
            reg.backward()
            optim.step()
            
        print(cost)

In [0]:
lg = LogisticRegression(input_dimensions=2)
lg.train(x,y)
w,b = lg.w, lg.b

plt.scatter(x[:, 0], x[:, 1],
            color = colors[y].tolist(), marker='.')
xmin, xmax = xlim()

ymin = -(b+w[0][0]*xmin)/w[1][0]
ymax = -(b+w[0][0]*xmax)/w[1][0]

plt.plot([xmin,xmax],[ymin,ymax], color="black", linestyle='-', linewidth=2)